In [3]:
# Useful starting lines
%matplotlib inline
import importlib.util
spec = importlib.util.spec_from_file_location("implementations", "./implementations.py")
implementations = importlib.util.module_from_spec(spec)
spec.loader.exec_module(implementations)
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [4]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [5]:
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [6]:
#apply method
def get_w_loss(y,x,method,initial_w=0,max_iters=10,gamma=0.00000005,threshold=1e-08,logs=False,batch_size=1,lambda_=0.005, store=False):
    """applies the algorithm described by <method>. by default as the awaited behaviour for the project, but offers some possible enhancements (batch_size,threshold, logs, store)
    Args:
        y (numpy.ndarray): An array with shape (n,1)
        tx (numpy.ndarray): An array with shape (n,m)
        method (int): 1 -> least_squares_GD
                      2 -> stochastic_gradient_descent
                      3 -> least_squares
                      4 -> ridge_regression
                      5 -> logistic_regression_gradient_descent
        initial_w (Union[float, numpy.ndarray], optional): if a int is set w will start with an array. Defaults to 0.
        max_iters (int, optional): number of iterations (only for descent algorithms). Defaults to 10.
        gamma (float, optional): descent scale factor (only for descent algorithms). Defaults to 0.00000005.
        threshold (float, optional): threshold to prevent divergence in case of linear separability (I assume). Defaults to 1e-08.
        log (bool, optional): if True step by step logs are displayed (only for descent algorithms). Defaults to False.
        batch_size (int, optional): size of the sample on which to compute the partial gradient for stochastic gradient descent algorithm. Defaults to 1.
        lambda_ (float, optional): regularization factor with good values allows to limit overfitting and underfitting (only for ridge_regression and reg_logistic_regression). Defaults to 0.005.
        store (bool, optional): [description]. Defaults to False.
    Returns:
        [type]: [description]
    :raises ValueError: if method is not <6
    """
    initial_=np.array([initial_w]*len(x[0]))
    if(method==1):
        return implementations.least_squares_GD(y,x, initial_, max_iters, gamma, logs)
    elif(method==2): 
        return implementations.stochastic_gradient_descent(y,x, initial_, max_iters, gamma, batch_size, logs)
    elif(method==3):
        return implementations.least_squares(y,x)
    elif(method==4):
        return implementations.ridge_regression(y,x, lambda_=lambda_)
    elif(method==5):
        return implementations.logistic_regression_gradient_descent(y,x,initial_w,max_iters,gamma,threshold,logs,store)
    elif(method==6):
        #TODO: 
        return implementations.reg_logistic_regression()
    else:
        return ValueError


In [7]:
OUTPUT_PATH = '../data/submission.csv'
mls=[]
weightss=[]
#for i in range(-10,10,1):
#    ml,weights=get_w(y,tX,2,i,max_iters=1000,log=False)
#    mls.append(ml)
#    weightss.append(weights)
#print(np.where(mls==min(mls)))
#print(mls)
method=5
weights, loss=get_w_loss(y,tX,method,logs=True,max_iters=100, gamma=0.0000000000005)
print(str(weights) + str(loss))
#weights=weightss[np.where(mls==min(mls))[0][0]]

if(method==5):
    data = np.c_[np.ones((_.shape[0], 1)), tX_test]
else:
    data=tX_test
y_pred = predict_labels(weights, data)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

Current iteration=0, loss=173286.79514050332
[-6.17887673e-06  1.19943068e-03 -4.27148113e-04 -5.02966885e-04
 -3.21625290e-04  3.69603517e-03  3.08322172e-03  3.69992628e-03
 -1.37888379e-05 -1.38235309e-04 -1.02280267e-03 -1.09121753e-05
  3.13320666e-06  3.69976193e-03 -1.90593905e-04  3.74478870e-08
 -6.20976348e-08 -3.07730393e-04  1.09704313e-07 -1.76073141e-07
 -2.80658172e-04  1.93013786e-07 -1.34491625e-03 -7.17524472e-06
  2.20956967e-03  2.51972284e-03  2.51973971e-03  3.55257615e-03
  3.70057986e-03  3.70050357e-03 -5.24478360e-04]-2110586.6709912037


In [9]:
 print(y[0]) 


1.0
